In [ ]:
from time import sleep
import selenium
from parsel import Selector
from selenium import webdriver as wd
import csv
import json
import pandas as pd
import re
import unicodedata
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

In [ ]:
def next_page():
    try:
        next_page = browser.find_element_by_xpath('//a[@class = "ui_button nav next primary "]')
        next_page.click()
    except selenium.common.exceptions.NoSuchElementException:
        pass
    
browser = get_browser() 
browser.get('https://www.tripadvisor.com/Hotels-g28932-zfc3-Hawaii-Hotels.html')
sleep(2)
select_hotel = browser.find_element_by_xpath('//*[@id="property_87119"]')
select_hotel.click()
sleep(2)
window_before = browser.window_handles[0]
window_after = browser.window_handles[1]
browser.switch_to_window(window_after)
    

df = pd.DataFrame()
f = 0
rows = []
m = []
Hotel_Name = get_hotel_name()
Hotel_Rating = get_hotel_rating()
j = 0
while(f < 132):

    reviews_pane = browser.find_elements_by_xpath('//div[@class = "hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H"]')

    
    for i in range(5):  
        if reviews_pane[i].text.splitlines()[4] == "Read more" :
            try:
                read_more = reviews_pane[i].find_element_by_xpath('./div[2]/div[3]/div[1]/span/div/span[1]')       
                read_more.click()
            except selenium.common.exceptions.NoSuchElementException:
                read_more = reviews_pane[i].find_element_by_xpath('./div[3]/div[3]/div[1]/span/div/span[1]')       
                read_more.click()
        else :
            pass
        
        try:             
            try: 
                cust_rating = reviews_pane[i].find_element_by_xpath('./div[2]/div[1]/span') 
            except NoSuchElementException:
                cust_rating = reviews_pane[i].find_element_by_xpath('./div[3]/div[1]/span')
                
        except selenium.common.exceptions.NoSuchElementException:
            try:
                cust_rating = reviews_pane[i].find_element_by_xpath('./div[2]/div[2]/span')
            except selenium.common.exceptions.NoSuchElementException:
                pass
            
        if cust_rating.get_attribute('class') == 'tWzdVpH_ dnQ8TT87' :
            x = 5
        elif cust_rating.get_attribute('class') == 'tWzdVpH_ _7BtjwjjB':
            x = 4
        elif cust_rating.get_attribute('class') == 'tWzdVpH_ _17GUOtor':
            x = 3
        elif cust_rating.get_attribute('class') == 'tWzdVpH_ _1LNKWH49':
            x = 2
        elif cust_rating.get_attribute('class') == 'tWzdVpH_ _1Nb7S6iu':
            x = 1
        
        Cust_name = reviews_pane[i].find_element_by_xpath('./div[1]/div/div[2]/span/a').text
        Cust_review_date = reviews_pane[i].text.splitlines()[0]
        Cust_review_date = Cust_review_date.replace(Cust_name,'')
        Cust_review_date = Cust_review_date.replace('wrote a review ','')
        try:
            Cust_contri = reviews_pane[i].find_element_by_xpath('./div[1]/div/div[3]/span[2]/span/span').text
        except selenium.common.exceptions.NoSuchElementException:
            Cust_contri = reviews_pane[i].find_element_by_xpath('./div[1]/div/div[3]/span/span/span').text
        Review_title = reviews_pane[i].text.splitlines()[2]
        Review = reviews_pane[i].text.splitlines()[3]
        try:
            Stay_date = reviews_pane[i].find_element_by_class_name("hotels-review-list-parts-EventDate__event_date--CRXs4").text
        except selenium.common.exceptions.NoSuchElementException:
            Stay_date = 'Not Available'
        m.append([Cust_name,Cust_review_date,Cust_contri,Review_title,Review,Stay_date,x,Hotel_Name,Hotel_Rating])
        print(j+1)
        j = j+1

        
        
    next_page()
    sleep(3)
    f = f + 1 

df = pd.DataFrame(m,columns = ['Cust_name','Cust_review_date','Cust_contri','Review_title','Review','Stay_date','Rating','Hotel_Name','Hotel_Rating'])



In [ ]:
df.to_excel("Final_Data.xlsx")